As the second step of this tutorial, we will train an image model. This step can be run in parallel with Step 3 (training the text model).

This notebook was run on an AWS p3.2xlarge

# Octopod Image Model Training Pipeline

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

Note: for images, we use the MultiInputMultiTaskLearner since we will send in the full image and a center crop of the image.

In [4]:
from octopod import MultiInputMultiTaskLearner, MultiDatasetLoader
from octopod.vision.dataset import OctopodImageDataset
from octopod.vision.models import ResnetForMultiTaskClassification

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [5]:
TRAIN_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_train.csv')

In [6]:
VALID_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_valid.csv')

In [7]:
TRAIN_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_train.csv')

In [8]:
VALID_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [9]:
batch_size = 64

We use the `OctopodImageDataSet` class to create train and valid datasets for each task.

Check out the documentation for infomation about the transformations.

In [10]:
gender_train_dataset = OctopodImageDataset(
    x=TRAIN_GENDER_DF['image_urls'],
    y=TRAIN_GENDER_DF['gender_cat'],
    transform='train',
    crop_transform='train'
)
gender_valid_dataset = OctopodImageDataset(
    x=VALID_GENDER_DF['image_urls'],
    y=VALID_GENDER_DF['gender_cat'],
    transform='val',
    crop_transform='val'
)

season_train_dataset = OctopodImageDataset(
    x=TRAIN_SEASON_DF['image_urls'],
    y=TRAIN_SEASON_DF['season_cat'],
    transform='train',
    crop_transform='train'
)
season_valid_dataset = OctopodImageDataset(
    x=VALID_SEASON_DF['image_urls'],
    y=VALID_SEASON_DF['season_cat'],
    transform='val',
    crop_transform='val'
)

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [11]:
train_dataloaders_dict = {
    'gender': DataLoader(gender_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'season': DataLoader(season_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
}
valid_dataloaders_dict = {
    'gender': DataLoader(gender_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
    'season': DataLoader(season_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
}

The dictionary of dataloaders is then put into an instance of the Octopod `MultiDatasetLoader` class.

In [12]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

730

In [13]:
ValidLoader = MultiDatasetLoader(loader_dict=valid_dataloaders_dict, shuffle=False)
len(ValidLoader)

244

We need to create a dictionary of the tasks and the number of unique values so that we can create our model.

In [14]:
new_task_dict = {
    'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
    'season': TRAIN_SEASON_DF['season_cat'].nunique(),
}

In [15]:
new_task_dict

{'gender': 5, 'season': 4}

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Create Model and Learner
===

These are completely new tasks so we use `new_task_dict`. If we had already trained a model on some tasks, we would use `pretrained_task_dict`.

And since these are new tasks, we set `load_pretrained_renset=True` to use the weights from Torch.

In [17]:
model = ResnetForMultiTaskClassification(
    new_task_dict=new_task_dict,
    load_pretrained_resnet=True
)

You will likely need to explore different values in this section to find some that work
for your particular model.

In [18]:
loss_function = nn.CrossEntropyLoss()

lr_last = 1e-2
lr_main = 1e-4

optimizer = optim.Adam([
    {'params': model.resnet.parameters(), 'lr': lr_main},
    {'params': model.dense_layers.parameters(), 'lr': lr_last},
    {'params': model.new_classifiers.parameters(), 'lr': lr_last},
    
])

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [19]:
learn = MultiInputMultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict)

Train model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [20]:
learn.fit(
    num_epochs=10,
    loss_function=loss_function,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,gender_train_loss,gender_val_loss,gender_acc,season_train_loss,season_val_loss,season_acc,time
0.710134,0.587723,0.558538,0.425387,0.842621,0.912374,0.804276,0.663163,05:22
0.594605,0.565996,0.435231,0.346161,0.880446,0.807223,0.859252,0.661511,05:22
0.557713,0.510187,0.400228,0.341952,0.887651,0.767811,0.734610,0.694098,05:23
0.531409,0.496355,0.370840,0.301714,0.898795,0.745623,0.756002,0.679231,05:22
0.454953,0.442471,0.284211,0.265934,0.910954,0.682738,0.677969,0.723232,05:23
0.427078,0.446454,0.260034,0.271331,0.909040,0.649927,0.680066,0.724283,05:22
0.414545,0.458448,0.246945,0.287083,0.907914,0.638137,0.687045,0.726536,05:22
0.401254,0.444956,0.234169,0.266389,0.913543,0.624160,0.683162,0.725184,05:23
0.384518,0.444607,0.219407,0.268102,0.915119,0.604791,0.680062,0.730440,05:23
0.381601,0.450801,0.216376,0.273748,0.915119,0.602026,0.686988,0.734344,05:23


Epoch 4 best model saved with loss of 0.44247129835929516


Validate model
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [21]:
pred_dict = learn.get_val_preds(device)

In [22]:
pred_dict

{'gender': {'y_true': array([0., 2., 4., ..., 4., 2., 2.]),
  'y_pred': array([[9.70583081e-01, 3.90860997e-03, 1.77693740e-02, 7.35662552e-03,
          3.82295839e-04],
         [4.52358545e-05, 6.18357763e-06, 9.99141693e-01, 1.09081397e-04,
          6.97762414e-04],
         [1.07502425e-03, 6.15622289e-02, 1.11506842e-02, 4.91130129e-02,
          8.77098978e-01],
         ...,
         [1.22738214e-07, 5.45594521e-05, 4.20508741e-06, 4.68803449e-07,
          9.99940634e-01],
         [2.34877516e-04, 1.46071898e-06, 9.30457532e-01, 5.25180809e-03,
          6.40542805e-02],
         [4.02231701e-04, 8.23197115e-06, 9.98965979e-01, 1.17490272e-04,
          5.05982141e-04]])},
 'season': {'y_true': array([0., 0., 2., ..., 2., 3., 3.]),
  'y_pred': array([[6.54517949e-01, 9.78528988e-04, 3.37896675e-01, 6.60690200e-03],
         [1.42849788e-01, 1.24642171e-03, 8.44931781e-01, 1.09719848e-02],
         [6.80897236e-02, 9.93176131e-04, 9.22244608e-01, 8.67245998e-03],
         ...

Save/Export Model
===

Once we are happy with our training we can save (or export) our model, using the `save` method (or `export`).

See the docs for the difference between `save` and `export`.

We will need the saved model later to use in the ensemble model

In [23]:
model.save(folder='/home/ubuntu/fashion_dataset/models/', model_id='IMAGE_MODEL1')

In [24]:
model.export(folder='/home/ubuntu/fashion_dataset/models/', model_id='IMAGE_MODEL1')

Now that we have an image model, we can move to `Step3_train_text_model`.